This notebook calculates the sediment contributing drainage area in the year 2025 for all of ResNet using the methods of Minear and Kondolf (xxxx).

In [16]:
#import packages
import pandas as pd
import numpy as np
import os
from datetime import datetime
import ast


pd.set_option('display.max_columns',None)

In [17]:
#load data

# ResNet File location and name
today = datetime.today().strftime('%Y%m%d')
# resnet_orig = pd.read_csv(f'Outputs/ResNet_{today}.csv')
resnet_orig = pd.read_csv(f'Outputs/ResNet_20250331.csv')

#load canadian dams for calculating
canada = pd.read_csv('Inputs/InputCanada.csv')

#output file location
out_folder = 'Outputs' 

In [18]:
resnet_orig.loc[resnet_orig.NID=='CO02005']

,Unnamed: 0,Dam_Name,ShortID,NID,GRanD_ID,IsSite,IsUSBR,IsUSACE,IsGRanD,State,OwnerTypes,PrimaryPur,IsRiverMth,delta,IsLock,yrc,yrr,yrc_source,MaxStor_m3,StorSource,Dam_Len_m,DamH_m,Longitude,Latitude,COMID,DivDASqKM,Moved,FromDam,ToDam,flag,countryOut,SiteTag,GRanDTag,RiverTag,DeltaTag


In [7]:
#conversions
convert1 = 1233.482 #converts m3 to ac-ft is convert1*AF=m3, or from m3 is m3/convert1=AF
convert2 = 2.59 #converts km2 and mi2 is convert2*mi2=km2 or from km2 is km2/convert2=mi2

In [8]:
#converts canada Batch_for to cell. Is this necessary?

#combine Canada and resnet
resnet = pd.concat([resnet_orig,canada])

resnet = resnet.sort_values(by='ShortID',ascending = True)

In [9]:
resnet.head()

,Unnamed: 0,Dam_Name,ShortID,NID,GRanD_ID,IsSite,IsUSBR,IsUSACE,IsGRanD,State,OwnerTypes,PrimaryPur,IsRiverMth,delta,IsLock,yrc,yrr,yrc_source,MaxStor_m3,StorSource,Dam_Len_m,DamH_m,Longitude,Latitude,COMID,DivDASqKM,Moved,FromDam,ToDam,flag,countryOut,SiteTag,GRanDTag,RiverTag,DeltaTag,GRAND_ID,PrimDamTyp,Reservoir,Year_First,Year_Last,Owner,RES_SED_No,CapOrig_m3,CapNew_m3,site_DA_km,yr_p,Capm3_p,USBRname,OCapm3_Rem,Batch_for,NIDStor_m3,GRanDCapm3,SA_m2,DA_km2,MaxQ_m3s,elev_m,NrX_Final,NrY_Final,LENGTHKM,Hydroseq,Pathlength,SLOPE,QA_MA,VA_MA,QC_MA,VC_MA,QE_MA,VE_MA,WBCOMID,D50_mm_,flagDA,flagTerm,flagHW,PermStorag
56575,869.0,CarsonRiver,-156.0,MOUTH_CarsonRiver,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,1.0,0.0,0.0,1700.0,0.0,Rivers,0.0,GDAT,0.0,0.0,-118.673030,39.710003,13069184,4336.1091,1.0,[341105.0],NaN,"[2, 1]",0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56572,872.0,TruckeeRiver,-155.0,MOUTH_TruckeeRiver,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,1.0,0.0,0.0,1700.0,0.0,Rivers,0.0,GDAT,0.0,0.0,-119.627058,40.196223,946050038,6176.5047,1.0,"[340918.0, 341194.0, 341111.0, 341115.0, 34111...",NaN,"[2, 1]",0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15820,41624.0,Myakka River,-154.0,MOUTH_MyakkaRiver,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,1.0,32.0,0.0,1700.0,0.0,Rivers,0.0,GDAT,0.0,0.0,-82.243484,26.995794,16841386,1478.7189,1.0,[293142.0],NaN,"[10, 1]",5,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15807,41637.0,WithlacoocheeRiver,-153.0,MOUTH_Withlacoochee,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,1.0,34.0,0.0,1700.0,0.0,Rivers,0.0,GDAT,0.0,0.0,-82.727732,29.015696,16944470,4596.9480,1.0,[40848.0],NaN,"[10, 1]",5,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
239,57205.0,KeysCreek,-152.0,MOUTH_KeysCreek,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,1.0,35.0,0.0,1700.0,0.0,Rivers,0.0,GDAT,0.0,0.0,-122.922582,38.221520,5329303,193.9473,1.0,[289611.0],NaN,"[10, 1]",42,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
#Convert strings to lists
resnet['FromDam'] = resnet['FromDam'].apply(lambda x: ast.literal_eval(x) if isinstance(x,str) else x)
resnet['ToDam'] = resnet['ToDam'].apply(lambda x: ast.literal_eval(x) if isinstance(x,str) else x)
resnet['flag'] = resnet['flag'].apply(lambda x: ast.literal_eval(x) if isinstance(x,str) else x)

In [ ]:
#Manually add major Canadian dams to the routing in the Columbia River basin for calculating sediment contributing drainage area

## Boundary Dam
resnet.loc[resnet.ShortID==117361, 'ToDam'] = 500005 #route to a Canadian dam, not directly to Grand Coulee
resnet.loc[resnet.ShortID==117361, 'GRanDTag'] = 500005

## Libby Dam
resnet.loc[resnet.ShortID==78196, 'ToDam'] = 500003 #route to a Canadian dam, not directly to Grand Coulee
resnet.loc[resnet.ShortID==78196, 'GRanDTag'] = 500003

## Grand Coulee Dam
resnet.loc[resnet.ShortID==117548, 'FromDam'] = resnet.loc[resnet.ShortID==117548, 'FromDam'].apply(
lambda x: [num for num in x if num not in {117361,78196}])) #remove Libby and Boundary ShortIDs from Coulee FromDam

resnet.loc[resnet.ShortID==117548, 'FromDam'] = resnet.loc[resnet.ShortID==117548, 'FromDam'].apply(
lambda x: x + [500002, 500004, 500006]) #Add Canadian dams to Coulee FromDam


In [45]:
#I think this entire cell can likely be deleted.

#changes yrr from 0/nan to 3001, ignore.

#changes nan todam to 0. again, can probably ignore.


UsageError: Cell magic `%%` not found.


In [ ]:
#Remove rivers from ToDam so only dams are routed to dams.

damstoriver = resnet.loc[resnet.ToDam<0]

while len(damstoriver)>0:
    for i in range(1,len(damstoriver)+1):
        #find a dam that goes to a river instead of another dam
        damlocationtofix = damstoriver[i] #index of the dam to fix
        river = resnet.loc[damlocationtofix,'ToDam'] #ShortID of the river it goes to
        riverloc = resnet.loc[resnet.ShortID==river] #location of the river
        rivertodam = resnet.loc[riverloc,'ToDam'] #the ToDam of the river
        resnet.loc[damlocationtofix,'ToDam'] = rivertodam #replace ToDam of target dam with ToDam of river
    damstoriver = resnet.loc[resnet.ToDam<0]
    
#Test this without loop version:
# # Identify dams that route to rivers
# damstoriver = resnet[resnet.ToDam < 0]

# # Create a mapping from rivers to their corresponding ToDam values
# river_to_dam_map = resnet.set_index("ShortID")["ToDam"]

# # Replace ToDam values of dams that currently route to rivers
# resnet.loc[damstoriver.index, "ToDam"] = damstoriver["ToDam"].map(river_to_dam_map).fillna(damstoriver["ToDam"])

In [ ]:
#  now that to dam has been replaced on real dams, remove to dam from rivers
resnet.loc[resnet.IsRiverMth==1,'ToDam'] = np.nan

#fix terminal dam flag after removing rivers
resnet.loc[resnet.ToDam.isna(),'flagTerm'] = 1
resnet.loc[resnet.IsRiverMth==1, 'flagTerm'] = np.nan


In [ ]:
#Ranking dams and assigning dam order (similar to stream order)

Rank = np.full(len(resNet['ShortID']),np.nan)

head = np.where(resnet.flagHW==1)[0]
rivers = np.where(resnet.IsRiverMth == 1)[0]
Rank[head] = 1
Rank[rivers] = 0

DAerrorNumber=0

i=1
ranknum= np.where(Rank == i)[0]

        damloc = np.where(dam.Hydroseq == d_s)[0][0] # Find which dam we reached
        
        ToDam[i] = dam.ShortID[damloc] # Update the ToDam column
        
while ranknum.size > 0:
    jmax =len(ranknum) #number of dams with rank of i
    for j in range(1, jmax + 1):
        damnow = ranknum[j] #current index
        
        if resnet.flagTerm[damnow] == 0: #if the dam is not a terminal dam
            damnowDA = resnet.DivDASqKM[damnow]
            todam = np.where(resnet.ShortID==resnet.ToDam[damnow])[0]
            todamDA = resnet.DivDASqKM[todam] #DA of the downstream dam

            #make sure downstream DA isn't bigger than upstream DA
            if todamDA<damnowDA:
                DAerrorNumber=DAerrorNumber+1;
                n=DAerrorNumber;
            else:
                Rank[todam]=Rank[damnow]+1;
                
    i=i+1
    ranknum=np.where(Rank==i)[0]

resnet['Rank'] = Rank


In [ ]:
#Get rid of rivers
resnet = resnet.loc[resnet['IsRiverMth'] != 1]


In [15]:
resnet.loc[resnet.yrc>2024,'yrc']

Series([], Name: yrc, dtype: float64)

In [ ]:
#For unreasonable or nonexistent yrc, replace with the 90th percentile of when dams were built; earlier than 1700 or later than 2024

# Identify valid values (1700 ≤ yrc ≤ 2024)
valid_yrc = resnet.loc[(resnet["yrc"] >= 1700) & (resnet["yrc"] <= 2024), "yrc"]

# Compute the 90th percentile from valid values
percentile_90 = np.percentile(valid_yrc, 90)

# Replace out-of-range values with the 90th percentile
resnet.loc[(resnet["yrc"] < 1700) | (resnet["yrc"] > 2024), "yrc"] = percentile_90

#could identify where we replaced yrc value, but probably not necessary here.

In [ ]:
# %% Fill in year prediction & capp- abby loop uses capp, so leave for now- this uses CAPP.  not sure what aaron needs.  may need to just set capp = capc or change code to have capc
# %sometimes a survey shows a growth in capacity- dam raise, or better survey technique etc. We have these data for some
# %Usace and usbr sites. here, the prediction year (yrp) is the newer survey.  Alternatively, sometimes there was no original survey
# % and the first survey was collected later. We will back calculate between yrp and yrc. 
# %if no yrp, then yrp=yrc
# % cap p is capacity at year p

#yrp is just yrc

#Just use MaxStor


In [ ]:
%create the timeseries
t = np.arange(1699, 2051)
numdam=numel(data.SID); #number of dams
numt=len(t); #length of time

#create empty variables
capcalc = np.full((len(resnet.ShortID), numt), np.nan) #this is a variable that would need to be stored in the structure
sedshed = np.full((len(resnet.ShortID), numt), np.nan) #this is the km2 area of the watershed that has sediment getting trapped in reservoir (so contribut DA * trap efficiency)
calctrap = np.full((len(resnet.ShortID),numt),0)
wsedshed = np.full((len(resnet.ShortID), 1), np.nan) #"effective sediment contributing DA"

origDA = np.full((len(resnet.ShortID), numt), np.nan) #original drainage area through time
for j in range(1,len(resnet.ShortID)+1):
    origDA[j,:] = resnet.DivDASqKM[j]

wSAatdam = np.full((len(resnet.ShortID), 1), np.nan) #Time-weighted sediment-contributing drainage area above reservoir X
AveTrap = np.full((len(resnet.ShortID), 1), np.nan) #time-weighted trap efficiency
wseddel = np.full((len(resnet.ShortID), 1), np.nan) #m3, total volume of sediment delivered to reservoir X between time 1 and time 2
wSDR = np.full((len(resnet.ShortID), 1), np.nan) #m3/yr, sediment delivery rate, mean volume of sediment delivered to reservoir X per year between time 1 and time 2
wSDRyield = np.full((len(resnet.ShortID), 1), np.nan) #m3/(km3*t), sediment yield, volume of sediment per km2 per year

sedDAtoDS = np.full((len(resnet.ShortID), numt), np.nan) #drainage area that moves downstream
SAatdam = origDA #sediment contributing drainage area upstream from reservoir X (does not include trap efficiency at reservoir X)

# All matlab below here for now

In [ ]:
# Trap efficiency

#for kappa: coarse (sand) = 1, medium (silt) = 0.1, fine (clay) = 0.046
#we use the design assumption for reservoirs of silt
kappa = np.full((len(resnet.ShortID), 1), 0.1)

#setting initial trap efficiency. Before and after the dam is in place it is zero. While the dam exists, calculate a value.
#This trap efficiency is static through time based on the initial TE.
for j in range(1,len(resnet.ShortID)+1):
    #assign trap efficiency as 0 before dam completion
    yrc = resnet.yrc[j]
    yrr = resnet.yrr[j]
    
    
    # Assuming 't' is a NumPy array and 'data' is a pandas DataFrame
    predam = np.where(t < yrc)[0]

    # Handling cases where yrr might be NaN
    if np.isnan(yrr):
        postdam = np.where(t >= yrc)[0]
        removed = np.array([])  # No dams removed
    else:
        postdam = np.where((t >= yrc) & (t < yrr))[0]
        removed = np.where(t >= yrr)[0]

    # Initialize arrays (assuming calctrap and capcalc exist)
    calctrap[j, predam] = 0
    capcalc[j, predam] = 0

    calctrap[j, removed] = 0
    capcalc[j, removed] = 0

    origDA[j, :] = resnet.DivDASqKM[j]

    calctrap[j, postdam] = 1 - 1. / (1 + kappa[j] * ((resnet.MaxStor[j] / convert1) / (resnet.DivDASqKM[j] / convert2)))

    capcalc[j, postdam] = resnet.MaxStor[j]

    # Since static, take the first trap efficiency for AveTrap
    AveTrap[j, 0] = calctrap[j, postdam[0]]
    
 

 for i in range(1,max(resnet.Rank)+1):
        ranknum = np.where(resnet.Rank == i)[0]
        jmax = len(ranknum)
        sedshed[ranknum,:] = SAatdam[ranknum,:] * calctrap[ranknum,:] #km2, this will calculate sedshed for the rank we are on
        sedDAtoDS[ranknum,:] = SAatdam[ranknum,:] - sedshed[ranknum,:] #km2, this is the volume moving downstream past the dam in a given year
        
        for j in range(1,jmax+1):
            val = ranknum[j]
            if resnet.flagTerm[val] == 0: #If it isn't a terminal dam, move the drainage area downstream
                todam = np.where(resnet.ShortID == resnet.ToDam[val])
                SAatdam[todam,:] = SAatdam[todam,:] - (resnet.DivDASqKM[val] - sedDAtoDS[val,:]) #Adjusts SA at dam for next time loop. Rank of this dam must be higher than dam it comes from
                
                #double check for any drainage area errors that can be caused by flow diversions
                DAerror = find(SAatdam[todam,:]<0)
                if len(DAerror)>0:
                    print('Drainage area error! Upstream drainage area is larger than downstream drainage area.')


In [ ]:
%% Save MLR TimesSeries Input...... you would want to change this file name for your export
%timeseries output, for MLR input
MLR_SAatdam_timeseries=NaN((numdam+1),(numt+1));
MLR_SAatdam_timeseries(1,2:end)=t;
MLR_SAatdam_timeseries(2:end,1)=data.SID;
MLR_SAatdam_timeseries(2:end,2:end)=SAatdam_TEyrc(:,:);
writematrix(MLR_SAatdam_timeseries,MLRfilename2,'Delimiter',',');


save MLRdata.mat

